# PyTorch - Fashion MNIST

## Resources
 

## Code Reference
 

In [35]:
import time
t0 = time.time()

In [36]:
import numpy as np
import matplotlib.pyplot as plt

In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [38]:
!pwd

/Users/reshamashaikh/ds/my_repos/pytorch_work


In [39]:
path = '/Users/reshamashaikh/ds/my_repos/pytorch_work/data_fashion/'

In [40]:
# """ A function that can read MNIST's idx file format into numpy arrays. (Yann LeCun)
#     This relies on the fact that the MNIST dataset consistently uses
#     unsigned char types with their data segments.
# """

# import struct

# import numpy as np

# def read_idx(filename):
#     with open(filename, 'rb') as f:
#         zero, data_type, dims = struct.unpack('>HBB', f.read(4))
#         shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
#         return np.fromstring(f.read(), dtype=np.uint8).reshape(shape)

In [41]:
# X_train = read_idx(path+'train-images-idx3-ubyte')
# y_train = read_idx(path+'train-labels-idx1-ubyte')

# X_test = read_idx(path+'t10k-images-idx3-ubyte')
# y_test = read_idx(path+'t10k-labels-idx1-ubyte')

In [42]:
# print(X_train.shape)
# print(y_train.shape)

# print(X_test.shape)
# print(y_test.shape)

In [43]:
#plt.hist(X_train, bins=np.arange(X_train.min(), X_train.max()+1))

In [44]:
#plt.hist(y_train, bins=np.arange(y_train.min(), y_train.max()+1))

In [45]:
## torchvision - download datasets
#https://pytorch.org/docs/stable/torchvision/datasets.html

In [46]:
# download and transform train dataset
train_loader = torch.utils.data.DataLoader(datasets.fashion_mnist('../fashion_mnist', 
                                                          download=True, 
                                                          train=True,
                                                          transform=transforms.Compose([
                                                              transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                              transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                          ])), 
                                           batch_size=10, 
                                           shuffle=True)

AttributeError: module 'torchvision.datasets' has no attribute 'fashion_mnist'

In [32]:
# download and transform test dataset
test_loader = torch.utils.data.DataLoader(datasets.MNIST('../fashion_mnist', 
                                                          download=True, 
                                                          train=False,
                                                          transform=transforms.Compose([
                                                              transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                              transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                          ])), 
                                           batch_size=10, 
                                           shuffle=True)

In [33]:
class CNNClassifier(nn.Module):
    """Custom module for a simple convnet classifier"""
    def __init__(self):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5)
        # default dropout is 0.5
        self.dropout = nn.Dropout2d(p=0.5)
        self.fc1 = nn.Linear(in_features=320, out_features=50)
        self.fc2 = nn.Linear(in_features=50, out_features=10)
    
    def forward(self, x):
        # input is 28x28x1
        # conv1(kernel=5, filters=10) 28x28x10 -> 24x24x10
        # max_pool(kernel=2) 24x24x10 -> 12x12x10
        
        # Do not be afraid of F's - those are just functional wrappers for modules form nn package
        # Please, see for yourself - http://pytorch.org/docs/_modules/torch/nn/functional.html
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        
        # conv2(kernel=5, filters=20) 12x12x20 -> 8x8x20
        # max_pool(kernel=2) 8x8x20 -> 4x4x20
        x = F.relu(F.max_pool2d(self.dropout(self.conv2(x)), kernel_size=2))
        
        # flatten 4x4x20 = 320
        x = x.view(-1, 320)
        
        # 320 -> 50
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        
        # 50 -> 10
        x = self.fc2(x)
        
        # transform to logits
        return F.log_softmax(x)

# create classifier and optimizer objects
clf = CNNClassifier()
opt = optim.SGD(clf.parameters(), lr=0.01, momentum=0.5)

loss_history = []
acc_history = []

def train(epoch):
    clf.train() # set model in training mode (need this because of dropout)
    
    # dataset API gives us pythonic batching 
    for batch_id, (data, label) in enumerate(train_loader):
        data = Variable(data)
        target = Variable(label)
        
        # forward pass, calculate loss and backprop!
        opt.zero_grad()
        preds = clf(data)
        loss = F.nll_loss(preds, target)
        loss.backward()
        loss_history.append(loss.data[0])
        opt.step()
        
        if batch_id % 1000 == 0:
            print(loss.data[0])

def test(epoch):
    clf.eval() # set model in inference mode (need this because of dropout)
    test_loss = 0
    correct = 0
    
    for data, target in test_loader:
        data = Variable(data, volatile=True) 
        target = Variable(target)
        
        output = clf(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    accuracy = 100. * correct / len(test_loader.dataset)
    acc_history.append(accuracy)
    print("---------------------------------------")
    print("Epoch %d" % epoch)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))

n_epochs = 3   
for epoch in range(0, n_epochs):
    #print("Epoch %d" % epoch)
    train(epoch)
    test(epoch)

run_time = time.time() - t0
print('Example run in : %.3f s' % run_time)
    
print("-----------------------")
print("the end")
print("-----------------------") 

/Users/reshamashaikh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/reshamashaikh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/reshamashaikh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:62: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


tensor(2.3800)
tensor(1.1453)
tensor(0.0637)
tensor(0.1996)
tensor(0.6800)
tensor(0.2454)


/Users/reshamashaikh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:70: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/Users/reshamashaikh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:74: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


---------------------------------------
Epoch 0

Test set: Average loss: 0.0962, Accuracy: 9707/10000 (97%)

tensor(0.2987)
tensor(0.3131)
tensor(0.1845)
tensor(0.2571)
tensor(0.1946)
tensor(0.0565)
---------------------------------------
Epoch 1

Test set: Average loss: 0.0695, Accuracy: 9782/10000 (97%)

tensor(0.3573)
tensor(0.3684)
tensor(0.3551)
tensor(0.0256)
tensor(0.2075)
tensor(0.6315)
---------------------------------------
Epoch 2

Test set: Average loss: 0.0691, Accuracy: 9792/10000 (97%)

Example run in : 115.743 s
-----------------------
the end
-----------------------


In [34]:
#globals()